In [6]:
## jupyter notebook to simulate the range of the nuclear recoils in a given decay chain

import numpy as np
import matplotlib.pyplot as plt
import alpha_recoil_sim as ar
import importlib


In [35]:
## make dictionary containing decay chain information for a given list of isotopes

path = "/Users/dcmoore/Library/CloudStorage/GoogleDrive-david.c.moore@yale.edu/My Drive/yale/uspheres/alpha_recoils_Grimm/Sphere_Recoils_MC"

iso_list = ['ac_225', 'pb_212', 'at_211', 'th_227', 'ra_223']

iso_dict = {}

for iso in iso_list:

    iso_dict[iso] = ar.parse_decay_chain(path + "/decay_data/" + iso + "_decay_chain.txt")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/dcmoore/Library/CloudStorage/GoogleDrive-david.c.moore@yale.edu/My Drive/yale/uspheres/alpha_recoils_Grimm/decay_data/ac_225_decay_chain.txt'